https://leetcode.com/problems/the-skyline-problem/
https://doi.org/10.1145/321906.321910

In [1]:
import collections
Building = collections.namedtuple("Building", ["left", "height", "right"])
Point2d = collections.namedtuple("Point2d", ["x", "y"])

In [2]:
def skyline_height(buildings, x):
    h = 0
    for b in buildings:
        if b.left <= x < b.right:
            h = max(h, b.height)
    return h    

Let us first present a simple quadratic time solution to the skyline problem

In [3]:
def add_point(points, p):
    assert p.y >= 0
    if points: 
        assert points[-1].x <= p.x
        if points[-1].x == p.x and points[-1].y < p.y:
            points[-1] = p
        elif points[-1].y != p.y:
            points.append(p)
    elif p.y > 0:
        points.append(p)

In [4]:
def skyline_quadratic(buildings):
    edges = [b.left for b in buildings] + [b.right for b in buildings]
    edges.sort()
    points = []
    for x in edges:
        add_point(points, Point2d(x, skyline_height(buildings, x)))
    return points    

In [5]:
from heapq import heappush, heappop

def skyline_height_heapq(buildings, *sorted_abscissae):    
    heights = []
    q = []
    i = 0
    for x in sorted_abscissae:
        while i < len(buildings) and buildings[i].left <= x:
            heappush(q, (- buildings[i].height, buildings[i].right))
            i += 1
        while q and q[0][1] <= x: heappop(q)
        heights.append(- q[0][0] if q else 0)       
    return heights

def skyline_heapq(buildings):
    buildings.sort(key=lambda b: b.left)
    edges = [b.left for b in buildings] + [b.right for b in buildings]
    edges.sort()
    points = []
    for e, h in zip(edges, skyline_height_heapq(buildings, *edges)) :
        add_point(points, Point2d(e, h))
    return points

In [6]:
def merge_skylines(points1, points2):
    dirty_points = []
    i1 = len(points1) - 1
    i2 = len(points2) - 1
    while i1 >= 0 and i2 >= 0:
        p1 = points1[i1]
        p2 = points2[i2]
        x = max(p1.x, p2.x)
        dirty_points.append(Point2d(x, max(p1.y, p2.y)))
        if p1.x == x: i1 -= 1
        if p2.x == x: i2 -= 1  
    points = points1[:i1 + 1] + points2[:i2 + 1]
    for p in reversed(dirty_points):
        add_point(points, p) 
    return points

In [7]:
def skyline_dc(buildings):
    l = len(buildings)
    if l == 1:
        b ,= buildings
        return [Point2d(b.left, b.height), Point2d(b.right, 0)]
    return merge_skylines(skyline_dc(buildings[:l//2]), skyline_dc(buildings[l//2:]))

In [8]:
def from_sorting_to_skyline(skyline, l):
    l = list(l)
    m = min(l) 
    M = max(l)    
    buildings = [Building(x - m, x - m + 1, M - m + i) for i, x in enumerate(l, 1)]
    points = skyline(buildings)
    return [p.x + m for p in points[:len(l)]]

print(from_sorting_to_skyline(skyline_quadratic, [5, 2, 3, 8, 1]))

[1, 2, 3, 5, 8]


In [9]:
test_set = '''
{
"consecutive": {
"in": [[0, 2, 1], [1, 3, 2], [2, 2, 3], [3, 2, 4]], 
"out": [[0, 2], [1, 3], [2, 2], [4, 0]]
}, 
"stairway to heaven": {
"in": [[1, 1, 5], [2, 2, 5], [3, 3, 5], [4, 4, 5]], 
"out": [[1, 1], [2, 2], [3, 3], [4, 4], [5, 0]]
}, 
"stairway from heaven": {
"in": [[1, 4, 2], [1, 3, 3], [1, 2, 4], [1, 1, 5]], 
"out": [[1, 4], [2, 3], [3, 2], [4, 1], [5, 0]]
}, 
"flat": {
"in": [[1, 1, 5], [2, 1, 6], [3, 1, 7], [4, 1, 8]], 
"out": [[1, 1], [8, 0]]
}, 
"tall": {
"in": [[1, 1, 2], [1, 3, 2], [1, 4, 2], [1, 2, 2]], 
"out": [[1, 4], [2, 0]]
}, 
"leetcode 1": {
"in": [[2, 10, 9], [3, 15, 7], [5, 12, 12], [15, 10, 20], [19, 8, 24]], 
"out": [[2, 10], [3, 15], [7, 12], [12, 0], [15, 10], [20, 8], [24, 0]]
}, 
"leetcode 2": {
"in": [[0, 3, 2], [2, 3, 5]], 
"out": [[0, 3], [5, 0]]
},
"Udi Manber": {
"in": [[1, 11, 5], [2, 6, 7], [3, 13, 9], [12, 7, 16], [14, 3, 25], [19, 18, 22], [23, 13, 29], [24, 4, 28]], 
"out": [[1, 11], [3, 13], [9, 0], [12, 7], [16, 3], [19, 18], [22, 3], [23, 13], [29, 0]]
}
}
'''
from json import loads
test_set = loads(test_set)

for v in test_set.values():
    d = v["in"]
    s = v["out"]
    for i, b in enumerate(d):
        d[i] = Building(*b)
    for i, p in enumerate(s):
        s[i] = Point2d(*p)
        
for test in test_set.values():
    assert skyline_quadratic(test["in"]) == test["out"]
    assert skyline_heapq(test["in"]) == test["out"]
    assert skyline_dc(test["in"]) == test["out"]

In [ ]:
prout